<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Цель: обучить модель классифицировать комментарии на позитивные и негативные на наборе данных с разметкой о токсичности правок.

Ход исследования: Данные представлены в файле toxic_comments.csv.

О качестве данных ничего неизвестно, поэтому перед проведением исследования понадобится обзор данных.

Таким образом, исследование пройдет в 3 этапа:

Открытие файлов с данными и их анализ;
Обучение моделей;
Общий вывод.

In [1]:
#загрузка библиотек
print("importing")
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import xgboost
import numpy as np
print("30%")
import requests
import numpy as np
from bs4 import BeautifulSoup
import os
import pandas as pd
import json
from datetime import datetime
import re
from tqdm.contrib.concurrent import thread_map
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets
print("70%")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import spacy
from tqdm import tqdm
from os import path
from pathlib import Path
from sklearn.pipeline import Pipeline
from nltk.probability import FreqDist
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("100%")

importing
30%
70%
100%


In [2]:
#поскольку файл долго лемматизируется, сохраним его и загрузим уже готовый файл для дальнейшей работы с ним.
def load(name, func):
    exist = os.path.exists(name)
    
    print(f"{'Reading' if exist else 'Executing'} {name}")
    
    if name.endswith("csv"):
        if exist:
            obj = pd.read_csv(name)
            index = list(obj['Unnamed: 0'])
            del obj['Unnamed: 0']
            obj.index = index
            return obj
        else:
            obj = func()
            obj.to_csv(name)
            return obj
    elif name.endswith('json'):
        if exist:
            with open(name, 'rb') as f:
                byt = f.read()
            return json.loads(byt.decode('utf-8'))
        else:
            obj = func()
            with open(name, 'wb') as f:
                f.write(json.dumps(obj).encode('utf-8'))
            return obj
    else:
        raise Exception('Undefined file type')

In [3]:
#загрузка файла
#на некоторых версиях on_bad_lines='warn'
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv',
               sep=',', error_bad_lines=False, names=['sentence', 'label'])

df['label'].unique()



array(['toxic', '0', '1'], dtype=object)

In [4]:
#поскольку в данных выше можно увидеть, что среди уникалных значений есть еще значения помимо разметки о токсичности правок
#присутствуют еще иные показатели, оставим только данные со значениями 0 и 1

df = df[(df['label'] == '0') | (df['label'] == '1')]
print(df.iloc[0])
df.head()

sentence    Explanation\nWhy the edits made under my usern...
label                                                       0
Name: 0.0, dtype: object


,sentence,label
0.0,Explanation\nWhy the edits made under my usern...,0
1.0,D'aww! He matches this background colour I'm s...,0
2.0,"Hey man, I'm really not trying to edit war. It...",0
3.0,"""\nMore\nI can't make any real suggestions on ...",0
4.0,"You, sir, are my hero. Any chance you remember...",0


In [5]:
# переведем разметку в тип int
df['label'] = df['label'].apply(int)
df['label'].unique()

array([0, 1])

In [6]:
#таким образом, видно, что в данных остались только значения 0 и 1. Посмотрим длину датасета
len(df)

159292

In [7]:
#напишем функцию для того, чтобы почистить текст, оставив только буквы из алфавита в нижнем регистре 
def clear_text(text):
    re_text = re.sub(r'[^a-zA-Z]', ' ', text)
    split_text = re_text.split()
    join_text = ' '.join(split_text)
    return join_text.lower()

In [8]:
#применим функцию к столбцу sentence
df['sentence'] = df['sentence'].apply(clear_text)

Сделаем лемматизацию

In [9]:
nlp = spacy.load('en_core_web_sm')

In [21]:
# #напишем функцию для лемматизации данных 
# def lemm_one(row):
#     return ' '.join([token.lemma_ for token in nlp(row)])
# def lemmatize(text):
#     return thread_map(lemm_one, text, max_workers=4)

In [10]:
#файл загружен на мой юпитер
#27:49
df['lemm'] = zload('lemm.json', lambda: lemmatize(df['sentence']))

In [11]:
#выделим целевой признак
X = df.drop(['label'], axis=1)
y = df['label']

In [12]:
y.value_counts()

0    143106
1     16186
Name: label, dtype: int64

Как видно из данных, количество 0 сильно отличается от количества 1. Данные не сбалансированы.

In [13]:
#разделим выборку на трейн и тест
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Сделаем TF-IDF. Выделим и удалим слова, которые не несут большой информативной нагрузки.

In [14]:
#!pip install nltk.corpus

In [15]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Применим TfidfVectorizer к тестовой и трейновой выборке

In [16]:
stop_words = set(stopwords.words('english'))
tf_idf = TfidfVectorizer(stop_words = stop_words)

In [17]:
X_train_tfidf = tf_idf.fit_transform(X_train['lemm'])
X_test_tfidf = tf_idf.transform(X_test['lemm'])

In [81]:
words = X['sentence']
y.value_counts()
index_1 = y[y==0].index

***Вывод:***  
В данных можно увидеть, что среди уникалных значений есть были значения помимо разметки о токсичности правок, в ходе анализа удалили лишние данные.
Провели лемматизацию и сделали TF-IDF. Выделили и удалили слова, которые не несли большой информативной нагрузки.

## Обучение

Обучим модель

In [34]:
pipelines = []
params = []
names = []

In [35]:
pipeline_tree = Pipeline([('tree', None)])  
   
pipeline_lg = Pipeline([('lg', None)])         

pipeline_catboost = Pipeline([('cat', None)])                          

In [36]:
from sklearn.utils.class_weight import compute_class_weight
 
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [37]:
param_tree = [
    {'tree': (RandomForestClassifier(random_state=1, class_weight = 'balanced'),),
     'tree__n_estimators': range(1, 20, 10),
     'tree__max_depth': range(1, 20, 10),
     }
]


param_lg = [
    {'lg': (LogisticRegression(random_state=1, class_weight = 'balanced'),),
     }
]

param_catboost = [
     {'cat': (CatBoostClassifier(random_state=1, verbose=False, class_weights = class_weights),),
     'cat__learning_rate': np.logspace(-3, -1, 1),
     'cat__iterations': range(25, 100, 100),
     'cat__depth': range(5,10,5),
      }
 ]

In [38]:

names = ['TREE', 'LG', 'CAT_BOOST']

In [39]:


pipelines.append(pipeline_tree)
pipelines.append(pipeline_lg)
pipelines.append(pipeline_catboost)


params.append(param_tree)

params.append(param_lg)

params.append(param_catboost)



In [40]:
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

In [41]:
results = {}
for pipeline, param_grid, name in tqdm(zip(pipelines, params, names)):   
    gs = GridSearchCV(pipeline,
                  param_grid,
                  scoring=f1,
                  n_jobs=-1,
                  cv=3)

    gs.fit(X_train_tfidf, y_train)  
     
     #top models table
    print('Top ' + name + ' models:')
    results_df = pd.DataFrame(gs.cv_results_)
    results_df = results_df.sort_values(by=["rank_test_score"])
    display(results_df[['params', 'mean_test_score', 'mean_fit_time', 'rank_test_score']]
            .head(5)
            .rename(columns={'mean_test_score': 'cv_score'})
            .rename(columns={'rank_test_score': 'rank_cv_score'}))

    results[name] = gs     




0it [00:00, ?it/s]

Top TREE models:


,params,cv_score,mean_fit_time,rank_cv_score
3,{'tree': RandomForestClassifier(class_weight='...,0.550635,1.856176,1
0,{'tree': RandomForestClassifier(class_weight='...,0.505681,0.120870,2
2,{'tree': RandomForestClassifier(class_weight='...,0.286925,0.260259,3
1,{'tree': RandomForestClassifier(class_weight='...,0.238381,0.325428,4


1it [00:09,  9.22s/it]/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Top LG models:


...

,params,cv_score,mean_fit_time,rank_cv_score
0,{'lg': LogisticRegression(class_weight='balanc...,0.85935,40.22418,1


2it [02:57, 102.99s/it]

Top CAT_BOOST models:


,params,cv_score,mean_fit_time,rank_cv_score
0,{'cat': <catboost.core.CatBoostClassifier obje...,0.768955,55.309938,1


3it [07:02, 140.70s/it]


****Вывод:****   
Как видно из данных, лучше всего себя показала модель LogisticRegression. Проведем проверку модели на тестовых данных.

## Выводы

In [42]:
preds = [results[x].best_estimator_.predict(X_test_tfidf) for x in names]

In [43]:
#выведем результат
scores = {name:f1_score(y_test, pred) for name, pred in zip(names, preds)}
scores

{'TREE': 0.32510030090270814,
 'LG': 0.7590705258866693,
 'CAT_BOOST': 0.5665415102211097}

*****Вывод:*****  
Как видно из данных, лучше всего себя показала модель LogisticRegression.